In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib as plt # data visualization

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


## Scopo del notebook
Lo scopo di questo progetto è quello di iniziare ad ottenere familiarità con i concetti e gli strumenti relativi al Machine Learning a alla sua applicazione pratica. 
Nello specifico, l'obiettivo sarà quello di addestrare un modello di classificazione in grado di determinare se un preciso passeggero è sopravvissuto o meno all'affondo del titanic, sulla base dei dati a disposizione nel dataset e riguardanti età, ricchezza e stato sociale.

Tale modello verrà addestrato sui dati contenuti nel training set train.csv e successivamente testato su un testset apposito, sprovvisto delle label riguardanti la sopravvivenza dei passeggeri.

Se necessario e rilevante, nel corso del notebook, verranno effettuati anche dei tentativi di feature engineering, al fine di creare nuove feature in grado di descrivere i dati.


In [2]:
# importo i dataset dall'input del notebook
trainData = pd.read_csv("../input/titanic/train.csv");
testData = pd.read_csv("../input/titanic/test.csv");

In [3]:
# svolgo una prima analisi del dataset tramite le funzioni di pandas che mi danno informazioni sulla sua composizione
# consideriamo ora solamente il dataset di training
trainData

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [4]:
# ottengo alcune informazioni statistiche sul dataset
trainData.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [5]:
# ottengo informazioni relative ad ogni colonna del dataset per quanto riguarda il tipo di dato 
trainData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


Possiamo subito notare come il dataset sia abbastanza completo in termini di dati, infatti abbiamo dei missing values solamente per quanto riguarda l'età di alcuni passeggeri e per quanto riguarda il numero di cabina. Questo ultimo fatto potrebbe essere più preoccupante in quanto non potremo sapere direttamente in che punto della nave erano collocati i passeggeri per i quali non è fornito il valore del numero di cabina.

Dal valore medio dell'attributo Survived, che vale 1 se il passeggero è sopravvissuto e 0 se è deceduto, posso subito constatare che la maggior parte dei passeggeri della nave non è riuscito a salvarsi.

Continuiamo ora con delle operazioni di preprocessing.

# PRE- PROCESSING

## Missing Values
E' molto importante rilevare ed occuparsi dei missing values all'interno del dataset, infatti la loro presenza potrebbe dare fastidio al training degli algoritmi di machine learning o potrebbe falsare i risultati ottenuti. 
Esistono perciò delle tecniche in grado di eliminarli oppure di trovare per essi un valore coerente.

Iniziamo con l'analisi dei missing values per quanto riguarda l'attributo "IMBARCATO", in quanto abbiamo solamente 2 missing values e siamo quindi di fronte ad una situazione più semplice.

In [6]:
# accedo solamente ai dati che hanno un valore nullo per l'attributo in questione
trainData.loc[pd.isnull(trainData["Embarked"])]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
61,62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN
829,830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN


Possiamo vedere come le due istanze che hanno un valore nullo nell'attributo che riguarda l'imbarco hanno lo stesso valore sia per quanto riguarda il prezzo sia per quanto riguarda la cabina assegnata. Le due passeggere, entrambe donne, però sono diverse in quanto hanno riportato un nome diverso. 
Ragionando,non possiamo concludere niente riguardo al posto in cui esse si sono imbarcate, perciò decidiamo di **lasciare un valore nullo**.

Continuiamo ora con i missing values riguardati l'età.

In [7]:
trainData.loc[pd.isnull(trainData["Age"])]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
17,18,1,2,"Williams, Mr. Charles Eugene",male,NaN,0,0,244373,13.0000,NaN,S
19,20,1,3,"Masselmani, Mrs. Fatima",female,NaN,0,0,2649,7.2250,NaN,C
26,27,0,3,"Emir, Mr. Farred Chehab",male,NaN,0,0,2631,7.2250,NaN,C
28,29,1,3,"O'Dwyer, Miss. Ellen ""Nellie""",female,NaN,0,0,330959,7.8792,NaN,Q
...,...,...,...,...,...,...,...,...,...,...,...,...
859,860,0,3,"Razi, Mr. Raihed",male,NaN,0,0,2629,7.2292,NaN,C
863,864,0,3,"Sage, Miss. Dorothy Edith ""Dolly""",female,NaN,8,2,CA. 2343,69.5500,NaN,S
868,869,0,3,"van Melkebeke, Mr. Philemon",male,NaN,0,0,345777,9.5000,NaN,S
878,879,0,3,"Laleff, Mr. Kristo",male,NaN,0,0,349217,7.8958,NaN,S


Anche per quanto riguarda l'età non possiamo dedurre niente dagli altri dati a nostra disposizione. Forse potremmo provare a dare un valore medio sulla base del valore medio degli altri passeggeri con una classe, una tariffa e un titolo simile. Sarebbe comunque una stima troppo approssimativa, in quanto sarebbe praticamente impossibile trovare il giusto valore nel continuo. Quindi passiamo oltre e analizziamo i dati mancanti per quanto riguarda la cabina.

In [8]:
noCabin = trainData.loc[pd.isnull(trainData["Cabin"])]
noCabin.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,687.000000,687.000000,687.000000,529.000000,687.000000,687.000000,687.000000
mean,443.208151,0.299854,2.639010,27.555293,0.547307,0.365357,19.157325
std,259.215905,0.458528,0.589602,13.472634,1.207492,0.827106,28.663343
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,214.500000,0.000000,2.000000,19.000000,0.000000,0.000000,7.877100
50%,441.000000,0.000000,3.000000,26.000000,0.000000,0.000000,10.500000
75%,664.500000,1.000000,3.000000,35.000000,1.000000,0.000000,23.000000
max,891.000000,1.000000,3.000000,74.000000,8.000000,6.000000,512.329200


Da questi dati possiamo notare la tendenza a non registrare la cabina solamente per quanto riguarda i passeggeri di seconda e sopratutto terza classe. Questo potrebbe far pensare ai passeggeri di classi sociali più basse, e quindi ovviamente più poveri e paganti un prezzo minore.
Anche in questo caso non appare evidente un modo per trovare i valori mancanti.

Decidiamo quindi di procedere in questo modo per quanto riguarda i missing values:
- Per quanto riguarda l'atrributo Embarked utilizzeremo un valore scelto casualmente, in questo caso S
- Per quanto riguarda il valore di Age cercheremo di utilizzare il valore medio di altri passeggeri in simili condizioni
- Per quanto riguarda l'attributo Cabin, decidiamo di rimuoverlo dal dataset, in quanto ha troppi missing values 

In [9]:
# questo comando permette di "fillare" i missing values con i valori che vengono passati come parametro
trainData = trainData.fillna(value = {"Embarked": "S"})

In [10]:
# eliminiamo la colonna relativa alla cabina
trainData = trainData.drop('Cabin',axis=1)
trainData

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S
...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C


In [11]:
# decidiamo di fillare i missing values in questo punto con il valore mediano dell'età degli altri passeggeri
trainData = trainData.fillna(trainData['Age'].median())
trainData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          891 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Embarked     891 non-null    object 
dtypes: float64(2), int64(5), object(4)
memory usage: 76.7+ KB


Abbiamo ora ottenuto un dataset completo in cui non abbiamo più missing values e possiamo procedere con le fasi successive del pre-processing dei dati.

## Text Preprocessing
Continuo ora la mia operazione di preprocessing applicando delle trasformazioni agli attributi testuali, per esempio trasformando la stringa del biglietto in modo da tenere solamente il numero.

In [12]:
# analizziamo prima la struttura dei biglietti
print(pd.unique(trainData["Ticket"]))

['A/5 21171' 'PC 17599' 'STON/O2. 3101282' '113803' '373450' '330877'
 '17463' '349909' '347742' '237736' 'PP 9549' '113783' 'A/5. 2151'
 '347082' '350406' '248706' '382652' '244373' '345763' '2649' '239865'
 '248698' '330923' '113788' '347077' '2631' '19950' '330959' '349216'
 'PC 17601' 'PC 17569' '335677' 'C.A. 24579' 'PC 17604' '113789' '2677'
 'A./5. 2152' '345764' '2651' '7546' '11668' '349253' 'SC/Paris 2123'
 '330958' 'S.C./A.4. 23567' '370371' '14311' '2662' '349237' '3101295'
 'A/4. 39886' 'PC 17572' '2926' '113509' '19947' 'C.A. 31026' '2697'
 'C.A. 34651' 'CA 2144' '2669' '113572' '36973' '347088' 'PC 17605' '2661'
 'C.A. 29395' 'S.P. 3464' '3101281' '315151' 'C.A. 33111' 'S.O.C. 14879'
 '2680' '1601' '348123' '349208' '374746' '248738' '364516' '345767'
 '345779' '330932' '113059' 'SO/C 14885' '3101278' 'W./C. 6608'
 'SOTON/OQ 392086' '343275' '343276' '347466' 'W.E.P. 5734' 'C.A. 2315'
 '364500' '374910' 'PC 17754' 'PC 17759' '231919' '244367' '349245'
 '349215' '35281' '

In [13]:
# ora elimino le parti alfabetiche
trainData["TicketNumber"] = pd.to_numeric(trainData["Ticket"].replace(".* {1}([0-9]*)$", "\\1", regex=True).replace("LINE","0")).astype("int64")
# con questa istruzione ho sostituito tutti i numeri o i caratteri speciali con uno spazio vuoto
trainData = trainData.drop("Ticket", axis=1) # rimuovo la vecchia colonna ticket
trainData

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,TicketNumber
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S,21171
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C,17599
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S,3101282
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S,113803
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S,373450
...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,13.0000,S,211536
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,30.0000,S,112053
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,28.0,1,2,23.4500,S,6607
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,30.0000,C,111369


Ora pongo gli attributi Sex, Pclass,Survived come Categorial ed elimino dal dataset il nome e l'ide dei passeggeri in quanto non sono rilevanti per la nostra analisi e portarseli dietro porta solo ad uno spreco di memoria.

In [14]:
trainData['Pclass']=trainData['Pclass'].astype('category')
trainData['Sex']=trainData['Sex'].astype('category')
trainData['Survived']=trainData['Survived'].astype('category')
trainData['Embarked']=trainData['Embarked'].astype('category')
trainData = trainData.drop(['Name','PassengerId'],axis=1)
trainData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   Survived      891 non-null    category
 1   Pclass        891 non-null    category
 2   Sex           891 non-null    category
 3   Age           891 non-null    float64 
 4   SibSp         891 non-null    int64   
 5   Parch         891 non-null    int64   
 6   Fare          891 non-null    float64 
 7   Embarked      891 non-null    category
 8   TicketNumber  891 non-null    int64   
dtypes: category(4), float64(2), int64(3)
memory usage: 38.9 KB
